In [21]:
import tools
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [22]:
import os

import os
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent




llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)



In [23]:
import os
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=2)

browser_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=(
    'Aufgaben:'
    '- Du bist ein Assistent, der Informationen über die Blütezeit von Pflanzen bereitstellt. Suche im Internet nach Informationen.'
    'Antworte in JSON und folge dem Schema: {"type": "object","properties": {"response": {"type": "string","enum": ["yes", "no", "no answer"]},reasoning": {"type": "string"}},"required": ["response", "reasoning"]}'
    ),
    name="browser_agent",
)

In [24]:
(
    'Aufgaben:'
    '- Du bist ein Assistent, der Informationen über die Blütezeit von Pflanzen bereitstellt. Suche im Internet nach Informationen.'
    'Antworte in JSON und folge dem Schema: {"type": "object","properties": {"response": {"type": "string","enum": ["yes", "no", "no answer"]},reasoning": {"type": "string"}},"required": ["response", "reasoning"]}'
    )

'Aufgaben:- Du bist ein Assistent, der Informationen über die Blütezeit von Pflanzen bereitstellt. Suche im Internet nach Informationen.Antworte in JSON und folge dem Schema: {"type": "object","properties": {"response": {"type": "string","enum": ["yes", "no", "no answer"]},reasoning": {"type": "string"}},"required": ["response", "reasoning"]}'

In [25]:
response = browser_agent.invoke({"role": "user", "messages": "Angenommen, heute ist der 30.07.2024 in K\u00f6ln \u2013 zeigt die Kastanie schon erste Bl\u00fcten?"})

In [26]:
response["messages"][-1].content


'{"response":"no answer","reasoning":"I attempted to search for current information on the chestnut tree (Kastanie) blooming status in Cologne as of July 30, 2024, but encountered a limitation with the search service, and therefore cannot provide a verified answer."}'

In [27]:
%pip install openai


Note: you may need to restart the kernel to use updated packages.


In [28]:
def invoke_browser_agent(query):
    while True:
        try:
            response = browser_agent.invoke(query)
            return response
        except Exception as e:
            print("No answer found, retrying...")
            continue

In [29]:
import json
from utils import get_tokens
#base_prompt = 'Antworte in JSON und folge dem Schema: { "type": "object", "properties": { "response": { "type": "boolean" }, "reasoning": { "type": "string" } }, "required": [ "response", "reasoning" ] }'
with open("datasets/dataset_850.json", "r") as file:
    dataset = json.load(file)


count = 0
responses = []
for data in dataset:
    sentence = data["sentence"]
    prompt = {"role": "user", "messages": sentence}

    try:
        messages = browser_agent.invoke(prompt)
    except Exception as e:
        responses.append({
        "id": data["id"],
        "plant": data["plant"],
        "city": data["city"],
        "date": data["date"],
        "sentence": sentence,
        "response": "error",
        "reasoning": str(e),
        "tokens": tokens
        })
        print(f"Error processing {data['id']}: {e}")
        continue
    
    response = messages["messages"][-1].content
    cleaned_response = response.strip("```json").strip("```").strip()
    response_json = json.loads(cleaned_response)
    tokens = get_tokens(messages)
    responses.append({
        "id": data["id"],
        "plant": data["plant"],
        "city": data["city"],
        "date": data["date"],
        "sentence": sentence,
        "response": response_json["response"],
        "reasoning": response_json["reasoning"],
        "tokens": tokens
    })
    count += 1
    print("Processed:", count, "/", len(dataset), "responses collected:", len(responses))

print(responses)

with open("results/llm_with_browser_850.json", "w") as file:
    json.dump(responses, file, indent=4)



Processed: 1 / 850 responses collected: 1
Processed: 2 / 850 responses collected: 2
Processed: 3 / 850 responses collected: 3
Processed: 4 / 850 responses collected: 4
Processed: 5 / 850 responses collected: 5
Processed: 6 / 850 responses collected: 6
Processed: 7 / 850 responses collected: 7
Processed: 8 / 850 responses collected: 8
Processed: 9 / 850 responses collected: 9
Processed: 10 / 850 responses collected: 10
Processed: 11 / 850 responses collected: 11
Processed: 12 / 850 responses collected: 12
Processed: 13 / 850 responses collected: 13
Processed: 14 / 850 responses collected: 14
Processed: 15 / 850 responses collected: 15
Processed: 16 / 850 responses collected: 16
Processed: 17 / 850 responses collected: 17
Processed: 18 / 850 responses collected: 18
Processed: 19 / 850 responses collected: 19
Processed: 20 / 850 responses collected: 20
Processed: 21 / 850 responses collected: 21
Processed: 22 / 850 responses collected: 22
Processed: 23 / 850 responses collected: 23
Proces

In [30]:
print(messages["messages"][-1].content)

{"response":"no answer","reasoning":"Der Zugriff auf die benötigten aktuellen Informationen über die Blütezeit von Raps in Stuttgart bis zum 18.05.2022 wurde aufgrund von Zugriffsbegrenzungen blockiert. Daher kann ich keine bestätigte Aussage dazu treffen."}
